# 🧪 Experiment Tracking Tutorial

Welcome to the fourth tutorial in our ML Pipeline series! In this notebook, we'll implement comprehensive experiment tracking using MLflow to manage our ML experiments like a professional data science team.

## 🎯 What You'll Learn
- Setting up MLflow experiment tracking
- Logging parameters, metrics, and artifacts
- Comparing experiments across multiple runs
- Managing model versions and lifecycle
- Creating experiment dashboards and reports
- Best practices for production MLOps

## 🏆 Learning Objectives
- **Track 20+ experiments** across different algorithms and parameters
- **Log comprehensive metrics** (accuracy, precision, recall, F1, R², RMSE)
- **Manage model artifacts** (models, plots, reports)
- **Compare experiment results** systematically
- **Implement model registry** for production deployment
- **Create experiment reports** for stakeholders

## 🛠️ Setup and Imports

In [ ]:
# =============================================================================
# UNIVERSAL SETUP - Works on all PCs and environments
# =============================================================================

import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Navigate to project root if we're in notebooks directory
if os.getcwd().endswith('notebooks'):
    os.chdir('..')
    print(f"📁 Changed to project root: {os.getcwd()}")
else:
    print(f"📁 Already in project root: {os.getcwd()}")

# Add src to Python path
src_path = os.path.join(os.getcwd(), 'src')
if src_path not in sys.path:
    sys.path.append(src_path)
    print(f"📦 Added to Python path: {src_path}")

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from datetime import datetime
import json
import uuid
import shutil

# MLflow imports
try:
    import mlflow
    import mlflow.sklearn
    from mlflow.tracking import MlflowClient
    print("✅ MLflow imported successfully")
    MLFLOW_AVAILABLE = True
except ImportError as e:
    print(f"⚠️ MLflow not available: {e}")
    print("💡 Install with: pip install mlflow")
    MLFLOW_AVAILABLE = False

# Scikit-learn imports
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, SVR
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, classification_report,
    mean_squared_error, mean_absolute_error, r2_score
)
import uuid  # Add this to your imports


# Configure plotting
try:
    plt.style.use('seaborn-v0_8')
except:
    plt.style.use('seaborn')  # Fallback for older versions

sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

print("✅ Setup completed successfully!")
if MLFLOW_AVAILABLE:
    print(f"🧪 MLflow version: {mlflow.__version__}")
print(f"📊 Scikit-learn version: {__import__('sklearn').__version__}")

## 🧪 MLflow Setup and Configuration

Let's set up MLflow for experiment tracking.

In [ ]:
class MLflowExperimentTracker:
    """Comprehensive MLflow experiment tracking system"""
    
    def __init__(self, tracking_uri=None, experiment_name=None):
        """Initialize MLflow experiment tracker"""
        self.mlflow_available = MLFLOW_AVAILABLE
        self.experiments = {}
        self.runs = {}
        
        if not self.mlflow_available:
            print("⚠️ MLflow not available - using mock tracking")
            self.mock_mode = True
            return
        
        self.mock_mode = False
        
        # Set up tracking URI
        if tracking_uri is None:
            mlruns_path = Path.cwd() / "mlruns"
            mlruns_path.mkdir(exist_ok=True)
            # Use proper file URI format for Windows
            tracking_uri = mlruns_path.as_uri()  # This creates proper file:// URI

        mlflow.set_tracking_uri(tracking_uri)
        
        print(f"🧪 MLflow tracking URI: {tracking_uri}")
        
        # Create experiments directory
        mlruns_dir = Path("mlruns")
        mlruns_dir.mkdir(exist_ok=True)
        
        print("✅ MLflow experiment tracker initialized!")
    
    def create_experiment(self, experiment_name, description=None):
        """Create a new MLflow experiment"""
        if self.mock_mode:
            print(f"🧪 [MOCK] Created experiment: {experiment_name}")
            self.experiments[experiment_name] = {'id': str(uuid.uuid4())[:8], 'runs': []}
            return self.experiments[experiment_name]['id']
        
        try:
            # Try to get existing experiment
            experiment = mlflow.get_experiment_by_name(experiment_name)
            if experiment:
                print(f"✅ Using existing experiment: {experiment_name} (ID: {experiment.experiment_id})")
                return experiment.experiment_id
        except:
            pass
        
        try:
            # Create new experiment
            experiment_id = mlflow.create_experiment(
                name=experiment_name,
                artifact_location=str(Path("mlruns") / "artifacts" / experiment_name)
            )
            print(f"✅ Created new experiment: {experiment_name} (ID: {experiment_id})")
            
            # Add description if provided
            if description:
                try:
                    self.client.update_experiment(experiment_id, description=description)
                except:
                    pass  # Description update not critical
            
            return experiment_id
            
        except Exception as e:
            print(f"⚠️ Error creating experiment: {e}")
            print("🔄 Using default experiment")
            return "0"  # Default experiment ID
    
    def start_run(self, experiment_name, run_name=None):
        """Start a new MLflow run"""
        if self.mock_mode:
            run_id = str(uuid.uuid4())[:8]
            print(f"🧪 [MOCK] Started run: {run_name or run_id}")
            return {'run_id': run_id, 'mock': True}
        
        try:
            # Set experiment
            mlflow.set_experiment(experiment_name)
            
            # Start run
            run = mlflow.start_run(run_name=run_name)
            print(f"🧪 Started MLflow run: {run_name or run.info.run_id[:8]}")
            return run
            
        except Exception as e:
            print(f"⚠️ Error starting run: {e}")
            return None
    
    def log_params(self, params, run=None):
        """Log parameters to MLflow"""
        if self.mock_mode:
            print(f"🧪 [MOCK] Logged params: {list(params.keys())[:3]}...")
            return
        
        try:
            mlflow.log_params(params)
        except Exception as e:
            print(f"⚠️ Error logging params: {e}")
    
    def log_metrics(self, metrics, step=None):
        """Log metrics to MLflow"""
        if self.mock_mode:
            print(f"🧪 [MOCK] Logged metrics: {list(metrics.keys())[:3]}...")
            return
        
        try:
            mlflow.log_metrics(metrics, step=step)
        except Exception as e:
            print(f"⚠️ Error logging metrics: {e}")
    
    def log_model(self, model, model_name, signature=None):
        """Log model to MLflow"""
        if self.mock_mode:
            print(f"🧪 [MOCK] Logged model: {model_name}")
            return
        
        try:
            mlflow.sklearn.log_model(
                model, 
                model_name,
                signature=signature
            )
        except Exception as e:
            print(f"⚠️ Error logging model: {e}")
    
    def log_artifact(self, artifact_path, artifact_name=None):
        """Log artifact to MLflow"""
        if self.mock_mode:
            print(f"🧪 [MOCK] Logged artifact: {artifact_name or artifact_path}")
            return
        
        try:
            if artifact_name:
                mlflow.log_artifact(artifact_path, artifact_name)
            else:
                mlflow.log_artifact(artifact_path)
        except Exception as e:
            print(f"⚠️ Error logging artifact: {e}")
    
    def end_run(self):
        """End current MLflow run"""
        if self.mock_mode:
            print(f"🧪 [MOCK] Ended run")
            return
        
        try:
            mlflow.end_run()
        except Exception as e:
            print(f"⚠️ Error ending run: {e}")

# Initialize the experiment tracker
tracker = MLflowExperimentTracker()

# Create main experiments
experiments_config = {
    "Titanic_Classification": "Titanic passenger survival prediction experiments",
    "Housing_Regression": "Boston housing price prediction experiments",
    "Model_Comparison": "Cross-dataset model comparison experiments",
    "Hyperparameter_Tuning": "Hyperparameter optimization experiments"
}

print("\n🧪 Creating MLflow experiments...")
for exp_name, description in experiments_config.items():
    tracker.create_experiment(exp_name, description)

print(f"\n✅ MLflow setup completed! Created {len(experiments_config)} experiments.")

## 📥 Load Training Data

Let's load our engineered features and trained models from previous tutorials.

In [ ]:
def load_experiment_data():
    """Load data for experiment tracking"""
    print("📥 Loading experiment data...")
    
    data = {}
    
    # Try to load engineered features
    feature_paths = {
        'titanic': ['data/features/titanic_features.csv', 'data/raw/titanic.csv'],
        'housing': ['data/features/housing_features.csv', 'data/raw/housing.csv']
    }
    
    for dataset_name, paths in feature_paths.items():
        loaded = False
        for path in paths:
            if Path(path).exists():
                try:
                    df = pd.read_csv(path)
                    data[dataset_name] = df
                    print(f"✅ Loaded {dataset_name} data from {path}: {df.shape}")
                    loaded = True
                    break
                except Exception as e:
                    print(f"⚠️ Error loading {path}: {e}")
                    continue
        
        if not loaded:
            print(f"❌ Could not load {dataset_name} data")
    
    # Try to load existing trained models
    models_dir = Path("trained_models")
    if models_dir.exists():
        model_files = list(models_dir.glob("*.joblib"))
        print(f"📦 Found {len(model_files)} existing trained models")
        
        # Load model registry if available
        registry_file = models_dir / "model_registry.json"
        if registry_file.exists():
            try:
                with open(registry_file, 'r') as f:
                    model_registry = json.load(f)
                data['model_registry'] = model_registry
                print(f"✅ Loaded model registry with {len(model_registry)} models")
            except Exception as e:
                print(f"⚠️ Error loading model registry: {e}")
    
    return data

# Load the data
experiment_data = load_experiment_data()

# Display summary
print(f"\n📊 Experiment Data Summary:")
for key, value in experiment_data.items():
    if isinstance(value, pd.DataFrame):
        print(f"   {key}: {value.shape} DataFrame")
    elif isinstance(value, list):
        print(f"   {key}: {len(value)} items")
    else:
        print(f"   {key}: {type(value).__name__}")

## 🤖 Comprehensive Experiment Runner

Let's create a comprehensive system to run and track multiple experiments.

In [ ]:
class ComprehensiveExperimentRunner:
    """Run and track comprehensive ML experiments"""
    
    def __init__(self, tracker):
        self.tracker = tracker
        self.experiment_results = {}
        
        # Define model configurations for experiments
        self.classification_configs = {
            'RandomForest_Basic': {
                'model': RandomForestClassifier(random_state=42),
                'params': {'n_estimators': 100, 'max_depth': 10}
            },
            'RandomForest_Tuned': {
                'model': RandomForestClassifier(random_state=42),
                'params': {'n_estimators': 200, 'max_depth': 15, 'min_samples_split': 5}
            },
            'LogisticRegression_Basic': {
                'model': LogisticRegression(random_state=42, max_iter=1000),
                'params': {'C': 1.0, 'solver': 'lbfgs'}
            },
            'LogisticRegression_Tuned': {
                'model': LogisticRegression(random_state=42, max_iter=1000),
                'params': {'C': 10.0, 'solver': 'liblinear'}
            },
            'SVM_Basic': {
                'model': SVC(random_state=42, probability=True),
                'params': {'C': 1.0, 'kernel': 'rbf'}
            },
            'SVM_Tuned': {
                'model': SVC(random_state=42, probability=True),
                'params': {'C': 10.0, 'kernel': 'rbf', 'gamma': 'scale'}
            }
        }
        
        self.regression_configs = {
            'RandomForest_Basic': {
                'model': RandomForestRegressor(random_state=42),
                'params': {'n_estimators': 100, 'max_depth': 10}
            },
            'RandomForest_Tuned': {
                'model': RandomForestRegressor(random_state=42),
                'params': {'n_estimators': 200, 'max_depth': 15, 'min_samples_split': 5}
            },
            'LinearRegression': {
                'model': LinearRegression(),
                'params': {}
            },
            'SVR_Basic': {
                'model': SVR(),
                'params': {'C': 1.0, 'kernel': 'rbf'}
            },
            'SVR_Tuned': {
                'model': SVR(),
                'params': {'C': 10.0, 'kernel': 'rbf', 'gamma': 'scale'}
            }
        }
    
    def prepare_data(self, data, target_col, test_size=0.2):
        """Prepare data for experiments"""
        # Handle missing values
        df = data.copy()
        
        # Fill missing values
        for col in df.columns:
            if df[col].dtype in ['object', 'category']:
                if col != target_col:
                    df[col] = pd.Categorical(df[col]).codes
            else:
                df[col].fillna(df[col].median(), inplace=True)
        
        # Separate features and target
        X = df.drop([target_col], axis=1)
        y = df[target_col]
        
        # Handle infinite values
        X = X.replace([np.inf, -np.inf], np.nan)
        X = X.fillna(X.median())
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=42,
            stratify=y if len(y.unique()) < 20 else None
        )
        
        return X_train, X_test, y_train, y_test, X.columns.tolist()
    
    def run_classification_experiments(self, data, target_col='Survived', experiment_name='Titanic_Classification'):
        """Run comprehensive classification experiments"""
        print(f"🧪 Running classification experiments for {experiment_name}...")
        
        # Prepare data
        X_train, X_test, y_train, y_test, feature_names = self.prepare_data(data, target_col)
        
        results = []
        
        for config_name, config in self.classification_configs.items():
            print(f"\n🔄 Running {config_name}...")
            
            # Start MLflow run
            run = self.tracker.start_run(experiment_name, f"{config_name}_{datetime.now().strftime('%H%M%S')}")
            
            try:
                # Set model parameters
                model = config['model'].__class__(**config['params'], random_state=42)
                
                # Train model
                model.fit(X_train, y_train)
                
                # Make predictions
                y_pred = model.predict(X_test)
                y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
                
                # Calculate metrics
                metrics = {
                    'accuracy': accuracy_score(y_test, y_pred),
                    'precision': precision_score(y_test, y_pred, average='weighted', zero_division=0),
                    'recall': recall_score(y_test, y_pred, average='weighted', zero_division=0),
                    'f1_score': f1_score(y_test, y_pred, average='weighted', zero_division=0)
                }
                
                if y_proba is not None:
                    try:
                        metrics['roc_auc'] = roc_auc_score(y_test, y_proba)
                    except:
                        metrics['roc_auc'] = 0.0
                
                # Cross-validation
                cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
                metrics['cv_mean'] = cv_scores.mean()
                metrics['cv_std'] = cv_scores.std()
                
                # Log to MLflow
                log_params = {
                    'model_type': config_name,
                    'algorithm': model.__class__.__name__,
                    'train_size': len(X_train),
                    'test_size': len(X_test),
                    'n_features': len(feature_names),
                    **config['params']
                }
                
                self.tracker.log_params(log_params)
                self.tracker.log_metrics(metrics)
                self.tracker.log_model(model, f"model_{config_name.lower()}")
                
                # Create and log confusion matrix plot
                self.create_confusion_matrix_plot(y_test, y_pred, config_name)
                
                # Store results
                result = {
                    'config_name': config_name,
                    'model': model,
                    'metrics': metrics,
                    'params': config['params'],
                    'run_id': run.info.run_id if run and not self.tracker.mock_mode else 'mock_' + str(uuid.uuid4())[:8]
                }
                results.append(result)
                
                print(f"   ✅ {config_name}: Accuracy = {metrics['accuracy']:.4f}")
                
            except Exception as e:
                print(f"   ❌ {config_name} failed: {e}")
            
            finally:
                self.tracker.end_run()
        
        self.experiment_results[experiment_name] = results
        return results
    
    def run_regression_experiments(self, data, target_col='MEDV', experiment_name='Housing_Regression'):
        """Run comprehensive regression experiments"""
        print(f"🧪 Running regression experiments for {experiment_name}...")
        
        # Prepare data
        X_train, X_test, y_train, y_test, feature_names = self.prepare_data(data, target_col)
        
        results = []
        
        for config_name, config in self.regression_configs.items():
            print(f"\n🔄 Running {config_name}...")
            
            # Start MLflow run
            run = self.tracker.start_run(experiment_name, f"{config_name}_{datetime.now().strftime('%H%M%S')}")
            
            try:
                # Set model parameters
                if config['params']:
                    model = config['model'].__class__(**config['params'])
                    if hasattr(model, 'random_state'):
                        model.random_state = 42
                else:
                    model = config['model']
                
                # Train model
                model.fit(X_train, y_train)
                
                # Make predictions
                y_pred = model.predict(X_test)
                
                # Calculate metrics
                metrics = {
                    'r2_score': r2_score(y_test, y_pred),
                    'mse': mean_squared_error(y_test, y_pred),
                    'rmse': np.sqrt(mean_squared_error(y_test, y_pred)),
                    'mae': mean_absolute_error(y_test, y_pred)
                }
                
                # Cross-validation
                cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
                metrics['cv_r2_mean'] = cv_scores.mean()
                metrics['cv_r2_std'] = cv_scores.std()
                
                # Log to MLflow
                log_params = {
                    'model_type': config_name,
                    'algorithm': model.__class__.__name__,
                    'train_size': len(X_train),
                    'test_size': len(X_test),
                    'n_features': len(feature_names),
                    **config['params']
                }
                
                self.tracker.log_params(log_params)
                self.tracker.log_metrics(metrics)
                self.tracker.log_model(model, f"model_{config_name.lower()}")
                
                # Create and log prediction plot
                self.create_regression_plot(y_test, y_pred, config_name)
                
                # Store results
                result = {
                    'config_name': config_name,
                    'model': model,
                    'metrics': metrics,
                    'params': config['params'],
                    'run_id': run.info.run_id if run and not self.tracker.mock_mode else 'mock_' + str(uuid.uuid4())[:8]
                }
                results.append(result)
                
                print(f"   ✅ {config_name}: R² = {metrics['r2_score']:.4f}")
                
            except Exception as e:
                print(f"   ❌ {config_name} failed: {e}")
            
            finally:
                self.tracker.end_run()
        
        self.experiment_results[experiment_name] = results
        return results
    
    def create_confusion_matrix_plot(self, y_true, y_pred, model_name):
        """Create and save confusion matrix plot"""
        try:
            plt.figure(figsize=(8, 6))
            cm = confusion_matrix(y_true, y_pred)
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
            plt.title(f'Confusion Matrix - {model_name}')
            plt.xlabel('Predicted')
            plt.ylabel('Actual')
            
            # Save plot
            plot_path = f"confusion_matrix_{model_name.lower()}.png"
            plt.savefig(plot_path, dpi=150, bbox_inches='tight')
            plt.close()
            
            # Log to MLflow
            self.tracker.log_artifact(plot_path)
            
            # Clean up
            if Path(plot_path).exists():
                Path(plot_path).unlink()
                
        except Exception as e:
            print(f"⚠️ Error creating confusion matrix plot: {e}")
    
    def create_regression_plot(self, y_true, y_pred, model_name):
        """Create and save regression prediction plot"""
        try:
            plt.figure(figsize=(8, 6))
            plt.scatter(y_true, y_pred, alpha=0.6)
            plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
            plt.xlabel('Actual Values')
            plt.ylabel('Predicted Values')
            plt.title(f'Actual vs Predicted - {model_name}')
            
            # Save plot
            plot_path = f"regression_plot_{model_name.lower()}.png"
            plt.savefig(plot_path, dpi=150, bbox_inches='tight')
            plt.close()
            
            # Log to MLflow
            self.tracker.log_artifact(plot_path)
            
            # Clean up
            if Path(plot_path).exists():
                Path(plot_path).unlink()
                
        except Exception as e:
            print(f"⚠️ Error creating regression plot: {e}")

# Initialize experiment runner
experiment_runner = ComprehensiveExperimentRunner(tracker)
print("✅ Comprehensive experiment runner initialized!")
print(f"📊 Classification configs: {len(experiment_runner.classification_configs)}")
print(f"📊 Regression configs: {len(experiment_runner.regression_configs)}")

## 🚢 Run Titanic Classification Experiments

Let's run comprehensive experiments on the Titanic dataset and track everything with MLflow.

In [ ]:
# Run Titanic classification experiments
if 'titanic' in experiment_data:
    print("🚢 TITANIC CLASSIFICATION EXPERIMENTS")
    print("=" * 50)
    
    titanic_results = experiment_runner.run_classification_experiments(
        experiment_data['titanic'], 
        target_col='Survived',
        experiment_name='Titanic_Classification'
    )
    
    print(f"\n✅ Titanic experiments completed! Ran {len(titanic_results)} experiments.")
    
    # Display results summary
    print("\n📊 Titanic Results Summary:")
    print("-" * 60)
    print(f"{'Model':<25} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1-Score':<10}")
    print("-" * 60)
    
    for result in sorted(titanic_results, key=lambda x: x['metrics']['accuracy'], reverse=True):
        metrics = result['metrics']
        print(f"{result['config_name']:<25} {metrics['accuracy']:<10.4f} "
              f"{metrics['precision']:<10.4f} {metrics['recall']:<10.4f} {metrics['f1_score']:<10.4f}")
    
    # Find best model
    best_titanic = max(titanic_results, key=lambda x: x['metrics']['accuracy'])
    print(f"\n🏆 Best Titanic Model: {best_titanic['config_name']} (Accuracy: {best_titanic['metrics']['accuracy']:.4f})")
    
else:
    print("⚠️ Titanic data not available for experiments")
    titanic_results = []

## 🏠 Run Housing Regression Experiments

Now let's run comprehensive experiments on the Housing dataset.

In [ ]:
# Run Housing regression experiments
if 'housing' in experiment_data:
    print("🏠 HOUSING REGRESSION EXPERIMENTS")
    print("=" * 50)
    
    housing_results = experiment_runner.run_regression_experiments(
        experiment_data['housing'], 
        target_col='MEDV',
        experiment_name='Housing_Regression'
    )
    
    print(f"\n✅ Housing experiments completed! Ran {len(housing_results)} experiments.")
    
    # Display results summary
    print("\n📊 Housing Results Summary:")
    print("-" * 60)
    print(f"{'Model':<25} {'R² Score':<10} {'RMSE':<10} {'MAE':<10}")
    print("-" * 60)
    
    for result in sorted(housing_results, key=lambda x: x['metrics']['r2_score'], reverse=True):
        metrics = result['metrics']
        print(f"{result['config_name']:<25} {metrics['r2_score']:<10.4f} "
              f"{metrics['rmse']:<10.4f} {metrics['mae']:<10.4f}")
    
    # Find best model
    best_housing = max(housing_results, key=lambda x: x['metrics']['r2_score'])
    print(f"\n🏆 Best Housing Model: {best_housing['config_name']} (R²: {best_housing['metrics']['r2_score']:.4f})")
    
else:
    print("⚠️ Housing data not available for experiments")
    housing_results = []

## 📊 Experiment Analysis and Visualization

Let's create comprehensive visualizations and analysis of our experiments.

In [ ]:
def create_experiment_visualizations(experiment_results):
    """Create comprehensive experiment visualizations"""
    print("📊 Creating experiment visualizations...")
    
    if not experiment_results:
        print("⚠️ No experiment results to visualize")
        return
    
    # Create subplots for each experiment
    n_experiments = len(experiment_results)
    fig, axes = plt.subplots(2, n_experiments, figsize=(6*n_experiments, 12))
    
    if n_experiments == 1:
        axes = axes.reshape(-1, 1)
    
    col_idx = 0
    
    for exp_name, results in experiment_results.items():
        if not results:
            continue
        
        # Determine if classification or regression
        is_classification = 'accuracy' in results[0]['metrics']
        
        # Extract data for plotting
        model_names = [r['config_name'] for r in results]
        
        if is_classification:
            primary_scores = [r['metrics']['accuracy'] for r in results]
            secondary_scores = [r['metrics']['f1_score'] for r in results]
            primary_label = 'Accuracy'
            secondary_label = 'F1-Score'
        else:
            primary_scores = [r['metrics']['r2_score'] for r in results]
            secondary_scores = [r['metrics']['rmse'] for r in results]
            primary_label = 'R² Score'
            secondary_label = 'RMSE'
        
        # Primary metric plot
        ax1 = axes[0, col_idx]
        bars1 = ax1.bar(range(len(model_names)), primary_scores, 
                       color=plt.cm.Set3(np.linspace(0, 1, len(model_names))))
        ax1.set_title(f'{exp_name.replace("_", " ")} - {primary_label}')
        ax1.set_xlabel('Models')
        ax1.set_ylabel(primary_label)
        ax1.set_xticks(range(len(model_names)))
        ax1.set_xticklabels([name.replace('_', '\n') for name in model_names], rotation=0, ha='center')
        
        # Add value labels
        for bar, score in zip(bars1, primary_scores):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height + max(primary_scores)*0.01,
                    f'{score:.3f}', ha='center', va='bottom', fontsize=9)
        
        # Secondary metric plot
        ax2 = axes[1, col_idx]
        bars2 = ax2.bar(range(len(model_names)), secondary_scores,
                       color=plt.cm.Set2(np.linspace(0, 1, len(model_names))))
        ax2.set_title(f'{exp_name.replace("_", " ")} - {secondary_label}')
        ax2.set_xlabel('Models')
        ax2.set_ylabel(secondary_label)
        ax2.set_xticks(range(len(model_names)))
        ax2.set_xticklabels([name.replace('_', '\n') for name in model_names], rotation=0, ha='center')
        
        # Add value labels
        for bar, score in zip(bars2, secondary_scores):
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height + max(secondary_scores)*0.01,
                    f'{score:.3f}', ha='center', va='bottom', fontsize=9)
        
        col_idx += 1
    
    plt.tight_layout()
    plt.show()
    
    # Create cross-validation comparison
    print("\n📊 Cross-Validation Performance Comparison:")
    
    fig, axes = plt.subplots(1, n_experiments, figsize=(6*n_experiments, 6))
    if n_experiments == 1:
        axes = [axes]
    
    col_idx = 0
    for exp_name, results in experiment_results.items():
        if not results:
            continue
        
        model_names = [r['config_name'] for r in results]
        
        # Get CV scores
        if 'cv_mean' in results[0]['metrics']:
            cv_means = [r['metrics']['cv_mean'] for r in results]
            cv_stds = [r['metrics']['cv_std'] for r in results]
            cv_label = 'CV Accuracy'
        else:
            cv_means = [r['metrics']['cv_r2_mean'] for r in results]
            cv_stds = [r['metrics']['cv_r2_std'] for r in results]
            cv_label = 'CV R² Score'
        
        ax = axes[col_idx]
        bars = ax.bar(range(len(model_names)), cv_means, yerr=cv_stds, capsize=5,
                     color=plt.cm.Pastel1(np.linspace(0, 1, len(model_names))))
        ax.set_title(f'{exp_name.replace("_", " ")} - {cv_label}')
        ax.set_xlabel('Models')
        ax.set_ylabel(cv_label)
        ax.set_xticks(range(len(model_names)))
        ax.set_xticklabels([name.replace('_', '\n') for name in model_names], rotation=0, ha='center')
        
        # Add value labels
        for bar, mean, std in zip(bars, cv_means, cv_stds):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + std + max(cv_means)*0.01,
                   f'{mean:.3f}±{std:.3f}', ha='center', va='bottom', fontsize=9)
        
        col_idx += 1
    
    plt.tight_layout()
    plt.show()

# Create visualizations
if experiment_runner.experiment_results:
    create_experiment_visualizations(experiment_runner.experiment_results)
else:
    print("⚠️ No experiment results available for visualization")

## 🏛️ Model Registry Management

Let's implement model registry functionality to manage our best models.

In [ ]:
class MLflowModelRegistry:
    """MLflow Model Registry management"""
    
    def __init__(self, tracker):
        self.tracker = tracker
        self.registered_models = {}
    
    def register_best_models(self, experiment_results):
        """Register best models from experiments"""
        print("🏛️ Registering best models to model registry...")
        
        for exp_name, results in experiment_results.items():
            if not results:
                continue
            
            # Find best model
            if 'accuracy' in results[0]['metrics']:
                best_result = max(results, key=lambda x: x['metrics']['accuracy'])
                metric_name = 'accuracy'
                metric_value = best_result['metrics']['accuracy']
            else:
                best_result = max(results, key=lambda x: x['metrics']['r2_score'])
                metric_name = 'r2_score'
                metric_value = best_result['metrics']['r2_score']
            
            # Create model name
            model_name = f"{exp_name}_Best_Model"
            
            # Register model (mock implementation)
            model_info = {
                'name': model_name,
                'version': 1,
                'stage': 'Staging',
                'run_id': best_result['run_id'],
                'config_name': best_result['config_name'],
                'best_metric': metric_name,
                'best_value': metric_value,
                'metrics': best_result['metrics'],
                'params': best_result['params'],
                'registered_at': datetime.now().isoformat(),
                'description': f"Best {exp_name.replace('_', ' ')} model with {metric_name}: {metric_value:.4f}"
            }
            
            self.registered_models[model_name] = model_info
            
            print(f"✅ Registered {model_name}:")
            print(f"   Algorithm: {best_result['config_name']}")
            print(f"   Best {metric_name}: {metric_value:.4f}")
            print(f"   Stage: Staging")
    
    def promote_to_production(self, model_name, min_accuracy=0.85, min_r2=0.65):
        """Promote model to production if it meets criteria"""
        if model_name not in self.registered_models:
            print(f"❌ Model {model_name} not found in registry")
            return False
        
        model_info = self.registered_models[model_name]
        
        # Check promotion criteria
        can_promote = False
        
        if model_info['best_metric'] == 'accuracy':
            can_promote = model_info['best_value'] >= min_accuracy
            threshold = min_accuracy
        elif model_info['best_metric'] == 'r2_score':
            can_promote = model_info['best_value'] >= min_r2
            threshold = min_r2
        
        if can_promote:
            model_info['stage'] = 'Production'
            model_info['promoted_at'] = datetime.now().isoformat()
            print(f"🚀 Promoted {model_name} to Production")
            print(f"   {model_info['best_metric']}: {model_info['best_value']:.4f} >= {threshold}")
            return True
        else:
            print(f"⚠️ {model_name} does not meet promotion criteria")
            print(f"   {model_info['best_metric']}: {model_info['best_value']:.4f} < {threshold}")
            return False
    
    def get_production_models(self):
        """Get all production models"""
        return {name: info for name, info in self.registered_models.items() 
                if info['stage'] == 'Production'}
    
    def create_model_registry_report(self):
        """Create comprehensive model registry report"""
        print("📄 Creating model registry report...")
        
        report_lines = []
        report_lines.append("# MLflow Model Registry Report\n\n")
        report_lines.append(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        # Summary
        total_models = len(self.registered_models)
        production_models = len(self.get_production_models())
        staging_models = total_models - production_models
        
        report_lines.append("## 📊 Registry Summary\n\n")
        report_lines.append(f"- **Total Models**: {total_models}\n")
        report_lines.append(f"- **Production Models**: {production_models}\n")
        report_lines.append(f"- **Staging Models**: {staging_models}\n\n")
        
        # Model details
        report_lines.append("## 🏛️ Registered Models\n\n")
        
        for model_name, info in self.registered_models.items():
            stage_emoji = "🚀" if info['stage'] == 'Production' else "🧪"
            report_lines.append(f"### {stage_emoji} {model_name}\n\n")
            report_lines.append(f"- **Stage**: {info['stage']}\n")
            report_lines.append(f"- **Algorithm**: {info['config_name']}\n")
            report_lines.append(f"- **Best Metric**: {info['best_metric']} = {info['best_value']:.4f}\n")
            report_lines.append(f"- **Registered**: {info['registered_at']}\n")
            
            if 'promoted_at' in info:
                report_lines.append(f"- **Promoted**: {info['promoted_at']}\n")
            
            report_lines.append(f"- **Description**: {info['description']}\n\n")
            
            # Performance metrics
            report_lines.append("**Performance Metrics:**\n")
            for metric, value in info['metrics'].items():
                report_lines.append(f"- {metric}: {value:.4f}\n")
            report_lines.append("\n")
        
        # Save report
        report_file = "model_registry_report.md"
        with open(report_file, 'w', encoding='utf-8') as f:
            f.writelines(report_lines)
        
        print(f"✅ Model registry report saved: {report_file}")
        return report_file

# Initialize model registry
model_registry = MLflowModelRegistry(tracker)

# Register best models
if experiment_runner.experiment_results:
    model_registry.register_best_models(experiment_runner.experiment_results)
    
    # Try to promote models to production
    print("\n🚀 Evaluating models for production promotion...")
    for model_name in model_registry.registered_models.keys():
        model_registry.promote_to_production(model_name)
    
    # Create registry report
    model_registry.create_model_registry_report()
    
    print(f"\n✅ Model registry management completed!")
    print(f"📊 Total registered models: {len(model_registry.registered_models)}")
    print(f"🚀 Production models: {len(model_registry.get_production_models())}")
    
else:
    print("⚠️ No experiment results available for model registry")

## 📈 Experiment Comparison and Analysis

Let's create comprehensive experiment comparison and analysis.

In [ ]:
def create_comprehensive_experiment_analysis(experiment_results, model_registry):
    """Create comprehensive experiment analysis"""
    print("📈 COMPREHENSIVE EXPERIMENT ANALYSIS")
    print("=" * 60)
    
    if not experiment_results:
        print("⚠️ No experiment results to analyze")
        return
    
    # Overall statistics
    total_experiments = sum(len(results) for results in experiment_results.values())
    total_datasets = len(experiment_results)
    
    print(f"📊 Overall Statistics:")
    print(f"   Datasets: {total_datasets}")
    print(f"   Total Experiments: {total_experiments}")
    print(f"   Registered Models: {len(model_registry.registered_models)}")
    print(f"   Production Models: {len(model_registry.get_production_models())}")
    
    # Performance analysis by dataset
    print(f"\n📊 Performance Analysis by Dataset:")
    print("-" * 60)
    
    analysis_data = []
    
    for exp_name, results in experiment_results.items():
        if not results:
            continue
        
        print(f"\n🎯 {exp_name.replace('_', ' ')}:")
        
        # Determine metric type
        is_classification = 'accuracy' in results[0]['metrics']
        primary_metric = 'accuracy' if is_classification else 'r2_score'
        
        # Extract performance data
        performances = [r['metrics'][primary_metric] for r in results]
        model_names = [r['config_name'] for r in results]
        
        # Statistics
        best_performance = max(performances)
        worst_performance = min(performances)
        avg_performance = np.mean(performances)
        std_performance = np.std(performances)
        
        best_model = results[performances.index(best_performance)]['config_name']
        worst_model = results[performances.index(worst_performance)]['config_name']
        
        print(f"   Best: {best_model} ({primary_metric}: {best_performance:.4f})")
        print(f"   Worst: {worst_model} ({primary_metric}: {worst_performance:.4f})")
        print(f"   Average: {avg_performance:.4f} ± {std_performance:.4f}")
        print(f"   Range: {worst_performance:.4f} - {best_performance:.4f}")
        
        # Store for overall analysis
        analysis_data.append({
            'dataset': exp_name,
            'task_type': 'Classification' if is_classification else 'Regression',
            'best_model': best_model,
            'best_performance': best_performance,
            'avg_performance': avg_performance,
            'std_performance': std_performance,
            'n_experiments': len(results)
        })
    
    # Algorithm performance analysis
    print(f"\n🤖 Algorithm Performance Analysis:")
    print("-" * 40)
    
    # Collect all algorithm performances
    algorithm_performances = {}
    
    for exp_name, results in experiment_results.items():
        is_classification = 'accuracy' in results[0]['metrics']
        primary_metric = 'accuracy' if is_classification else 'r2_score'
        
        for result in results:
            algo_name = result['config_name'].split('_')[0]  # Get base algorithm name
            if algo_name not in algorithm_performances:
                algorithm_performances[algo_name] = []
            algorithm_performances[algo_name].append(result['metrics'][primary_metric])
    
    # Analyze algorithm performance
    for algo, performances in algorithm_performances.items():
        avg_perf = np.mean(performances)
        std_perf = np.std(performances)
        n_experiments = len(performances)
        
        print(f"   {algo}: {avg_perf:.4f} ± {std_perf:.4f} ({n_experiments} experiments)")
    
    # Target achievement analysis
    print(f"\n🎯 Target Achievement Analysis:")
    print("-" * 35)
    
    targets = {
        'Titanic_Classification': {'target': 0.894, 'metric': 'accuracy'},
        'Housing_Regression': {'target': 0.681, 'metric': 'r2_score'}
    }
    
    achievements = []
    
    for exp_name, results in experiment_results.items():
        if exp_name in targets:
            target_info = targets[exp_name]
            target_value = target_info['target']
            metric_name = target_info['metric']
            
            # Find best performance
            best_result = max(results, key=lambda x: x['metrics'][metric_name])
            best_performance = best_result['metrics'][metric_name]
            
            achievement_pct = (best_performance / target_value) * 100
            status = "✅ ACHIEVED" if best_performance >= target_value else "⚠️ CLOSE" if achievement_pct >= 95 else "❌ NEEDS WORK"
            
            print(f"   {exp_name.replace('_', ' ')}:")
            print(f"     Target: {target_value:.3f}")
            print(f"     Achieved: {best_performance:.3f}")
            print(f"     Achievement: {achievement_pct:.1f}% - {status}")
            print(f"     Best Model: {best_result['config_name']}")
            
            achievements.append({
                'dataset': exp_name,
                'target': target_value,
                'achieved': best_performance,
                'achievement_pct': achievement_pct,
                'status': status,
                'best_model': best_result['config_name']
            })
    
    # Create summary DataFrame
    if analysis_data:
        analysis_df = pd.DataFrame(analysis_data)
        print(f"\n📊 Experiment Summary Table:")
        print(analysis_df.to_string(index=False))
    
    return analysis_data, achievements

# Run comprehensive analysis
if experiment_runner.experiment_results:
    analysis_data, achievements = create_comprehensive_experiment_analysis(
        experiment_runner.experiment_results, 
        model_registry
    )
else:
    print("⚠️ No experiment results available for analysis")
    analysis_data, achievements = [], []

## 📄 Generate Comprehensive Experiment Report

Let's create a comprehensive report of all our experiments and findings.

In [ ]:
def generate_comprehensive_experiment_report(experiment_results, model_registry, analysis_data, achievements):
    """Generate comprehensive experiment report"""
    print("📄 Generating comprehensive experiment report...")
    
    report_lines = []
    report_lines.append("# 🧪 MLflow Experiment Tracking Report\n\n")
    report_lines.append(f"**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    
    # Executive Summary
    total_experiments = sum(len(results) for results in experiment_results.values())
    total_datasets = len(experiment_results)
    successful_targets = len([a for a in achievements if 'ACHIEVED' in a['status']])
    
    report_lines.append("## 📊 Executive Summary\n\n")
    report_lines.append(f"- **Total Experiments Run**: {total_experiments}\n")
    report_lines.append(f"- **Datasets Analyzed**: {total_datasets}\n")
    report_lines.append(f"- **Models Registered**: {len(model_registry.registered_models)}\n")
    report_lines.append(f"- **Production Models**: {len(model_registry.get_production_models())}\n")
    report_lines.append(f"- **Targets Achieved**: {successful_targets}/{len(achievements)}\n\n")
    
    # Experiment Results by Dataset
    report_lines.append("## 🎯 Experiment Results by Dataset\n\n")
    
    for exp_name, results in experiment_results.items():
        if not results:
            continue
        
        report_lines.append(f"### {exp_name.replace('_', ' ')}\n\n")
        
        # Task type
        is_classification = 'accuracy' in results[0]['metrics']
        task_type = 'Classification' if is_classification else 'Regression'
        report_lines.append(f"**Task Type:** {task_type}\n\n")
        
        # Results table
        if is_classification:
            report_lines.append("| Rank | Model | Accuracy | Precision | Recall | F1-Score | CV Score |\n")
            report_lines.append("|------|-------|----------|-----------|--------|----------|----------|\n")
            
            sorted_results = sorted(results, key=lambda x: x['metrics']['accuracy'], reverse=True)
            for i, result in enumerate(sorted_results, 1):
                m = result['metrics']
                report_lines.append(f"| {i} | {result['config_name']} | {m['accuracy']:.4f} | "
                                   f"{m['precision']:.4f} | {m['recall']:.4f} | {m['f1_score']:.4f} | "
                                   f"{m['cv_mean']:.4f} |\n")
        else:
            report_lines.append("| Rank | Model | R² Score | RMSE | MAE | CV R² |\n")
            report_lines.append("|------|-------|----------|------|-----|-------|\n")
            
            sorted_results = sorted(results, key=lambda x: x['metrics']['r2_score'], reverse=True)
            for i, result in enumerate(sorted_results, 1):
                m = result['metrics']
                report_lines.append(f"| {i} | {result['config_name']} | {m['r2_score']:.4f} | "
                                   f"{m['rmse']:.4f} | {m['mae']:.4f} | {m['cv_r2_mean']:.4f} |\n")
        
        report_lines.append("\n")
        
        # Best model details
        best_result = sorted_results[0]
        report_lines.append(f"**🏆 Best Model:** {best_result['config_name']}\n\n")
        report_lines.append(f"**Parameters:**\n")
        for param, value in best_result['params'].items():
            report_lines.append(f"- {param}: {value}\n")
        report_lines.append("\n")
    
    # Target Achievement Analysis
    report_lines.append("## 🎯 Target Achievement Analysis\n\n")
    
    for achievement in achievements:
        status_emoji = "✅" if "ACHIEVED" in achievement['status'] else "⚠️" if "CLOSE" in achievement['status'] else "❌"
        report_lines.append(f"### {status_emoji} {achievement['dataset'].replace('_', ' ')}\n\n")
        report_lines.append(f"- **Target**: {achievement['target']:.3f}\n")
        report_lines.append(f"- **Achieved**: {achievement['achieved']:.3f}\n")
        report_lines.append(f"- **Achievement Rate**: {achievement['achievement_pct']:.1f}%\n")
        report_lines.append(f"- **Status**: {achievement['status']}\n")
        report_lines.append(f"- **Best Model**: {achievement['best_model']}\n\n")
    
    # Model Registry Summary
    report_lines.append("## 🏛️ Model Registry Summary\n\n")
    
    production_models = model_registry.get_production_models()
    
    if production_models:
        report_lines.append("### 🚀 Production Models\n\n")
        for model_name, info in production_models.items():
            report_lines.append(f"**{model_name}**\n")
            report_lines.append(f"- Algorithm: {info['config_name']}\n")
            report_lines.append(f"- Performance: {info['best_metric']} = {info['best_value']:.4f}\n")
            report_lines.append(f"- Promoted: {info.get('promoted_at', 'N/A')}\n\n")
    else:
        report_lines.append("No models currently in production.\n\n")
    
    # Recommendations
    report_lines.append("## 💡 Recommendations\n\n")
    
    # Generate recommendations based on results
    recommendations = []
    
    # Check if targets were achieved
    if successful_targets < len(achievements):
        recommendations.append("🎯 **Performance Improvement**: Some models did not meet target performance. Consider:")
        recommendations.append("   - Additional feature engineering")
        recommendations.append("   - Hyperparameter optimization")
        recommendations.append("   - Ensemble methods")
        recommendations.append("   - More training data")
    
    if len(production_models) == 0:
        recommendations.append("🚀 **Model Deployment**: No models are currently in production. Consider promoting the best performing models.")
    
    recommendations.append("📊 **Monitoring**: Implement model monitoring to track performance degradation in production.")
    recommendations.append("🔄 **Continuous Training**: Set up automated retraining pipelines for model updates.")
    recommendations.append("🧪 **A/B Testing**: Implement A/B testing framework for model comparison in production.")
    
    for rec in recommendations:
        report_lines.append(f"{rec}\n\n")
    
    # Next Steps
    report_lines.append("## 🚀 Next Steps\n\n")
    report_lines.append("1. **Model Deployment**: Deploy production-ready models using the deployment tutorial\n")
    report_lines.append("2. **Monitoring Setup**: Implement model monitoring and alerting\n")
    report_lines.append("3. **Performance Optimization**: Continue hyperparameter tuning for better results\n")
    report_lines.append("4. **Feature Engineering**: Explore additional feature engineering techniques\n")
    report_lines.append("5. **Ensemble Methods**: Combine best models for improved performance\n\n")
    
    # Technical Details
    report_lines.append("## 🔧 Technical Details\n\n")
    report_lines.append(f"- **MLflow Tracking URI**: {tracker.tracking_uri if hasattr(tracker, 'tracking_uri') else 'Mock Mode'}\n")
    report_lines.append(f"- **Experiments Created**: {len(experiments_config)}\n")
    report_lines.append(f"- **Total Runs**: {total_experiments}\n")
    report_lines.append(f"- **Artifacts Logged**: Confusion matrices, regression plots, model files\n")
    report_lines.append(f"- **Cross-Validation**: 5-fold CV used for all models\n\n")
    
    # Save report
    report_file = "comprehensive_experiment_report.md"
    with open(report_file, 'w', encoding='utf-8') as f:
        f.writelines(report_lines)
    
    print(f"✅ Comprehensive experiment report saved: {report_file}")
    return report_file

# Generate comprehensive report
if experiment_runner.experiment_results:
    report_file = generate_comprehensive_experiment_report(
        experiment_runner.experiment_results,
        model_registry,
        analysis_data,
        achievements
    )
    print(f"\n📄 Report generated: {report_file}")
else:
    print("⚠️ No experiment results available for report generation")

## 🎉 Congratulations!

You've successfully completed the experiment tracking tutorial! You now understand:

✅ **MLflow Setup**: Configured comprehensive experiment tracking  
✅ **Experiment Management**: Created and organized multiple experiments  
✅ **Parameter Logging**: Tracked all model parameters and hyperparameters  
✅ **Metrics Tracking**: Logged comprehensive performance metrics  
✅ **Artifact Management**: Saved models, plots, and reports  
✅ **Model Registry**: Implemented model versioning and lifecycle management  

### 🏆 What We Achieved

**🧪 Experiment Tracking:**
- **Total Experiments**: 10+ comprehensive experiments
- **Datasets**: Titanic (Classification) + Housing (Regression)
- **Algorithms**: Random Forest, Logistic Regression, SVM, Linear Regression, SVR
- **Metrics Logged**: Accuracy, Precision, Recall, F1, R², RMSE, MAE, CV scores
- **Artifacts**: Confusion matrices, regression plots, model files

**🏛️ Model Registry:**
- **Model Registration**: Best models automatically registered
- **Lifecycle Management**: Staging → Production promotion workflow
- **Performance Criteria**: Automated promotion based on performance thresholds
- **Model Metadata**: Complete model information and lineage

**📊 Analysis & Reporting:**
- **Performance Comparison**: Side-by-side model comparison
- **Target Achievement**: Tracking against performance goals
- **Comprehensive Reports**: Detailed experiment analysis
- **Recommendations**: Data-driven improvement suggestions

### 🔧 Key Techniques Mastered

1. **MLflow Integration**: Professional experiment tracking setup
2. **Systematic Experimentation**: Organized, reproducible experiments
3. **Performance Monitoring**: Comprehensive metrics tracking
4. **Model Lifecycle**: From experimentation to production
5. **Artifact Management**: Organized storage of models and plots
6. **Automated Analysis**: Data-driven model comparison and selection

### 🚀 Next Tutorial
In the final notebook (`05_model_deployment.ipynb`), we'll learn to:
- Deploy models to production environments
- Create REST APIs for model serving
- Implement Docker containerization
- Set up monitoring and logging
- Create web interfaces for predictions

### 💡 Practice Exercises
Try these exercises to reinforce your learning:
1. Add more algorithms (XGBoost, Neural Networks)
2. Implement automated hyperparameter optimization
3. Create custom metrics for domain-specific evaluation
4. Set up automated experiment scheduling

### 📁 Files Created
Your experiment tracking artifacts are saved as:
- `mlruns/` - MLflow experiment data and artifacts
- `comprehensive_experiment_report.md` - Detailed experiment analysis
- `model_registry_report.md` - Model registry status
- Various plots and visualizations

### 🎯 MLflow UI Access
To view your experiments in the MLflow UI:
```bash
mlflow ui --backend-store-uri file:///./mlruns
```
Then open: http://localhost:5000

Your experiments are now professionally tracked and ready for production deployment! 🎊

---

**🎯 Ready for Model Deployment?**  
Run: `jupyter notebook notebooks/05_model_deployment.ipynb`